<a href="https://colab.research.google.com/github/Mostofa-Najmus-Sakib/NLP-works/blob/main/Assisgnment_5/A5_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Topic Modeling with Word Embeddings, Tensorflow, and Keras

- We'll be using the [pymagnitude](https://github.com/plasticityai/magnitude) library

In [ ]:
# %pip install pymagnitude

     |████████████████████████████████| 5.4MB 7.0MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.143-cp37-cp37m-linux_x86_64.whl size=360622092 sha256=d30f1d1f856d9057c960bdfb6cdb82af6e634cb9b729a5e282cc1ac1aadc6ea4
  Stored in directory: /root/.cache/pip/wheels/bb/b2/16/f88eb50800667ffe23d509c7e2157923a08d10ed6d6410124f
Successfully built pymagnitude


In [ ]:
from pymagnitude import *

In [ ]:
# from google.colab import files
# uploaded = files.upload()

### You'll need to download embedding 'model' files manually

Start by downloading one of the following:

- [GloVe](http://magnitude.plasticity.ai/glove/medium/glove.6B.300d.magnitude)
- [word2vec](http://magnitude.plasticity.ai/word2vec/heavy/GoogleNews-vectors-negative300.magnitude)
- [fastText](http://magnitude.plasticity.ai/fasttext/light/wiki-news-300d-1M.magnitude)


In [ ]:
# change path to the name/path of the embedding file you donwloaded
path = 'glove.6B.300d.magnitude'

# vectors = Magnitude(path)
vectors = Magnitude('glove.6B.300d.magnitude')

In [ ]:
len(vectors)

400000

In [ ]:
vectors.dim # this is how big the vectors are for each word

300

In [ ]:
"cat" in vectors

True

In [ ]:
for key, vector in vectors[500:510]:
    print(key, vector[:3])

working [-0.0332886  0.0680554 -0.0059854]
community [-0.0431237 -0.0621079  0.0222967]
eight [-0.0645148  0.0757905 -0.0573475]
groups [-0.0194742  0.0416667 -0.0084207]
despite [ 0.0479242  0.02233   -0.0047541]
level [-0.0848005  0.1553303  0.0087196]
largest [-0.0233524  0.04329   -0.0260668]
whose [-0.0069965  0.0276343  0.0280235]
attacks [0.03818   0.0117073 0.0706972]
germany [ 0.0062351  0.0252957 -0.0618449]


In [ ]:
vectors.query("cat")[:3]

array([-0.0463976,  0.0525527, -0.007488 ], dtype=float32)

In [ ]:
vectors.query(["cat","dog"])[0][:3]

array([-0.0463976,  0.0525527, -0.007488 ], dtype=float32)

In [ ]:
vectors.distance("cat", "dog")

0.7979039

In [ ]:
vectors.distance("cat", "car")

1.3062327

In [ ]:
vectors.most_similar_to_given("cat", ["dog", "television", "laptop"]) 

'dog'

In [ ]:
vectors.doesnt_match(["breakfast", "cereal", "dinner", "lunch"])

'cereal'

In [ ]:
#vectors.most_similar("cat", topn = 5)

In [ ]:
#vectors.most_similar(positive = ["woman", "king"], negative = ["man"])

### Topic Modeling

- Given a document, determine the topic of the document
- For this task, we'll use the Brown corpus of texts accessible via NLTK

In [ ]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [ ]:
# from nltk.corpus import brown
# from collections import defaultdict
# import tqdm # tqdm displays a progress bar
# from tqdm import tqdm_notebook as tqdm

category_vectors = []

cats = brown.categories()
    
# for each category
for cat in cats:
    print(cat)
    # grab all of the documents
    for fileid in tqdm(brown.fileids(categories=[cat])):
        words = list(map(str.lower, brown.words(fileids=[fileid])))
        # grab all of the words, find their embedding, sum all embeddings
        word_sum = np.sum([vectors.query([w]) for w in words if w in vectors], axis=0) # why axis=0?
        # add the now summed embedding to the list for this category
        category_vectors.append((cat,word_sum))
    

adventure


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



belles_lettres



editorial



fiction


...


news



religion



reviews



romance



science_fiction


In [ ]:
import pandas as pd

keys,values=zip(*category_vectors) # unzip using a *

data = pd.DataFrame({'cat':keys,'vectors':values})

In [ ]:
data[:3]

,cat,vectors
0,adventure,"[[-62.71034, 23.958982, -9.739936, -54.190174,..."
1,adventure,"[[-54.73519, 19.858883, -8.82027, -59.00295, -..."
2,adventure,"[[-46.095287, 24.262121, -7.475177, -59.681107..."


In [ ]:
total = len(data)
total

500

#### compute the baselines

In [ ]:
print('random baseline {}'.format(1.0/len(cat)))

print('most common baseline?')
for cat in cats:
    print(cat, len(data[data.cat==cat])/total)

random baseline 0.06666666666666667
most common baseline?
adventure 0.058
belles_lettres 0.15
editorial 0.054
fiction 0.058
government 0.06
hobbies 0.072
humor 0.018
learned 0.16
lore 0.096
mystery 0.048
news 0.088
religion 0.034
reviews 0.034
romance 0.058
science_fiction 0.012


#### split the data into train/test

In [ ]:
test = data.sample(frac=0.1,random_state=200)
train = data.drop(test.index)

test.shape, train.shape 

((50, 2), (450, 2))

#### train a classifier

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.cat) 
X = [x[0] for x in train.vectors]
y = le.transform(train.cat)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clfr = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [ ]:
clfr.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### evaluate 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_y = le.transform(test.cat)
test_X = [x[0] for x in test.vectors]

score = accuracy_score(clfr.predict(test_X), test_y)

In [ ]:
print(path, score)

glove.6B.300d.magnitude 0.52


data/GoogleNews-vectors-negative300.magnitude 0.4
data/wiki-news-300d-1M.magnitude 0.56
data/glove.6B.300d.magnitude 0.52

## Preprocessing data for neural network


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder()
le.fit(data.cat) 
y = le.transform(train.cat).reshape(-1,1)
ohe.fit(y)

y = ohe.transform(y).todense()

X = np.array([x[0] for x in train.vectors])
X.shape, y.shape


((450, 300), (450, 15))

In [ ]:

from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam

### With a single dense hidden layer

In [ ]:
model = Sequential()
model.add(layers.Dense(32, input_dim = 300, activation='sigmoid'))
model.add(layers.Dense(15, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 32)                9632      
_________________________________________________________________
dense_61 (Dense)             (None, 15)                495       
Total params: 10,127
Trainable params: 10,127
Non-trainable params: 0
_________________________________________________________________


In [ ]:
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])

In [ ]:
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 0.21 and accuracy of 0.93 here

### With a single dense hidden layer (Changing number of nodes, activation functions

In [ ]:
model = Sequential()
model.add(layers.Dense(64, input_dim = 300, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs = 100, verbose = False)

In [ ]:
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])


In [ ]:
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 0.27 and accuracy of 0.93 here

In [ ]:
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 1.92 and accuracy of 0.87 here

### With additional hidden layers

In [ ]:
model = Sequential()
model.add(layers.Dense(128, input_dim = 300, activation='sigmoid'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_65 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_66 (Dense)             (None, 15)                975       
Total params: 47,759
Trainable params: 47,759
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs = 100, verbose = False)
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 0.16 and accuracy of 0.93 here

### Adding regularization (Dropout)

In [ ]:
model = Sequential()
model.add(layers.Dense(128, input_dim = 300, activation='sigmoid'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs = 100, verbose = False)
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 0.17 and accuracy of 0.93 here

### Adding regularization (Norm)

In [ ]:
from keras import regularizers

model = Sequential()
model.add(layers.Dense(128, input_dim = 300, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs = 100, verbose = False)
test_y = ohe.transform(le.transform(test.cat).reshape(-1, 1))
test_X = np.array([x[0] for x in test.vectors])
# loss, accuracy = model.evaluate(test_X, test_y, verbose = True)
# loss, accuracy

I had a loss of 0.21 and accuracy of 0.93 here



In [ ]:
le = preprocessing.LabelEncoder() 
ohe = preprocessing.OneHotEncoder() 

le.fit(data.cat) 
y = le.transform(train.cat).reshape(-1, 1) 
ohe.fit(y)
y = ohe.transform(y).todense()

X = np.array([x[0] for x in train.vectors])

X.shape, y.shape

test_y = le.transform(test.cat).reshape(-1, 1) 
ohe.fit(test_y)
test_y = ohe.transform(test_y).todense()

test_X = np.array([x[0] for x in test.vectors])

test_X.shape, test_y.shape

((50, 300), (50, 14))

In [ ]:
def final_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='sigmoid', kernel_regularizer='l2', input_shape=(300,)),
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer='l2'),
    tf.keras.layers.Dense(15, activation='softmax',kernel_regularizer='l2'),
    tf.keras.layers.Dense(15)
  ])

  model.compile(optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
model = None
model = final_model()
model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_74 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_75 (Dense)             (None, 15)                975       
_________________________________________________________________
dense_76 (Dense)             (None, 15)                240       
Total params: 47,999
Trainable params: 47,999
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,y, epochs=200)

Epoch 1/200
15/15 [==============================] - 1s 2ms/step - loss: 3.4260 - accuracy: 0.0849
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 2.6282 - accuracy: 0.0748
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 2.0333 - accuracy: 0.0840
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 1.6880 - accuracy: 0.0728
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 1.4466 - accuracy: 0.1213
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 1.3108 - accuracy: 0.1403
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 1.1278 - accuracy: 0.1067
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 1.0712 - accuracy: 0.1458
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 0.9903 - accuracy: 0.1487
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 0.9222 - accuracy: 0.1513
Epoch 11/

I had a really low accuracy rate here

#### What would you say is the neural network "learning"?

Ans: A neural network is tring to recognize underlying relationships in the given  set of data through a process that mimics the way the human brain operates. In this sense, neural networks refer to systems of neurons, either organic or artificial in nature. Neural networks can adapt to changing input; so the network generates the best possible result without needing to redesign the output criteria. 

The network is actually learning the mapping between a 300 dimensional vector (the summed up vector of every word embeddings vector of words from each category) and a 15 dimensional one hot encoded vector (that contains the class/category label). Throughout the training, the network is fed 450 such example pairs to learn a mapping function.

#### How does the depth or width of the network affect the training and the results?

Ans: Depth of the network (adding additional hidden layers in between input and output layer) improves the result. This is because the model become capable of holding longer non-linear relationships.
Though, adding to many hidden layers decreases the performance because of overfitting into the train data.

Also, width of the network (adding more neurons/nodes in each layer) seems to increase the performance in a certain level of adjustment.

In either way, the training time increases when depth/width is increased because the number of parameter increases proportionally which needs extra computational time.

#### As you made changes to the network, what do you notice about how parameters (network depth, number of nodes, learning rate, etc.) and how they interact with each other? We said that neural networks are learning non-convex problems, but what about finding the best parameters? Is that a convex problem?

Ans: I think this is a convex problem because while I tried to add too many depths/nodes/learning rate it actually become really slow. But we were also focusing on minimizing the loss 

#### What is regularization? Why is it important?

Ans: Regularization is a technique which makes slight modifications to the learning algorithm such that the model generalizes better. This in turn improves the model’s performance on the unseen data as well.

In other words this helps to resolve the overfitting (situation where the model performed exceptionally well on train data, but is not able to do good at test data) issue. In deep learning, it penalizes the weight matrices of the nodes resulting into a much simpler linear network and slight underfitting of the training data.

#### Which activation functions did you choose (besides logitistic/sigmoid)? For one of the activation functions you tried, spend some time learning about it. Whereas logistic/sigmoid maps from inputs to a probability between 0-1, what does the activation function you chose do?

Ans: I have used the sigmoid, relu and softmax function. The main reason to use sigmoid function is because it exists between (0 to 1). Therefore, it is especially used for models where we have to predict the probability as an output.

The rectified linear activation function or ReLU for short is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero.

The softmax function is a function that turns a vector of K real values into a vector of K real values that sum to 1.